In [3]:
import os
import cv2
from stl import mesh
import numpy as np
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

In [4]:
SRC = '/Users/fathe/OneDrive/Documents/UK/MFF/Thesis/input'
folders = [file for file in os.listdir(SRC) if file.startswith("GX0") and os.path.isdir(os.path.join(SRC, file))]

In [3]:
folders

['GX010988', 'GX010996', 'GX010998']

In [4]:
scan_paths = []
oio_paths = []
for folder in folders:
    scan_paths.append(
        os.path.join(folder, [file for file in os.listdir(os.path.join(SRC, folder)) if file.endswith("unrolled_final.stl")][0]))
    oio_paths.append(folder + '-oio.png')

In [5]:
scan_paths, oio_paths

(['GX010988\\Hnízdo_719_unrolled_final.stl',
  'GX010996\\Hnízdo_732_unrolled_final.stl',
  'GX010998\\Hnizdo_718_unrolled_final.stl'],
 ['GX010988-oio.png', 'GX010996-oio.png', 'GX010998-oio.png'])

In [55]:
oios = []
for path in oio_paths:
    oios.append(cv2.imread(os.path.join(SRC, path), cv2.IMREAD_UNCHANGED))

In [56]:
oios[0].shape

(8309, 19640)

In [57]:
scans = []
for scan_path in scan_paths:
    scans.append(mesh.Mesh.from_file(os.path.join(SRC, scan_path)))

In [63]:
def findBoundaries(mesh):
    # Initialize min and max values
    min_x = float('inf')
    max_x = float('-inf')
    min_z = float('inf')
    max_z = float('-inf')
    
    # Iterate through each vertex in the mesh
    for triangle in tqdm(mesh.vectors, total=len(mesh.vectors), desc='Finding boundaries'):
        for vertex in triangle:
            # Check for min and max x
            if vertex[0] < min_x:
                min_x = vertex[0]
            if vertex[0] > max_x:
                max_x = vertex[0]
            
            # Check for min and max z
            if vertex[2] < min_z:
                min_z = vertex[2]
            if vertex[2] > max_z:
                max_z = vertex[2]
    
    return min_x, max_x, min_z, max_z

In [70]:
for scan, oio, scan_path, folder in zip(scans, oios, scan_paths, folders):
    print(f'Starting pre-calculations for {scan_path}')
    
    height, width = oio.shape
    
    factor = 0.1
    height, width = int(height*factor), int(width*factor)
    
    print(f'Desired dimensions of depth map: {height, width}')

    depth_map = np.zeros((height, width), dtype=np.float32)
    
    min_x, max_x, min_z, max_z = findBoundaries(scan)
    
    print(f'Boundaries of scan mesh: {min_x, max_x, min_z, max_z}')
    
    x_scale = width/(max_x - min_x)
    z_scale = height/(max_z - min_z)
    
    print(f'Scaling between mesh and depth map: {x_scale, z_scale}')
    
    # Iterate over each triangle in the mesh
    for triangle in tqdm(scan.vectors, total=len(scan.vectors), desc=f'Computing depth map for {scan_path}'):
        # Project each triangle's vertices onto the image plane
        projected_points = []
        for vertex in triangle:
            # Calculate 2D projection (perspective projection)
            y = vertex[1]
            x = (vertex[0]-min_x) * x_scale
            z = (vertex[2]-min_z) * z_scale
            if 0 <= int(x) < width and 0 <= int(z) < height:
                projected_points.append((int(x), y, int(z)))
    
        # Fill the depth map
        for (x, y, z) in projected_points:
            depth_map[z, x] = max(depth_map[z, x], y)  # Store max depth (or average if desired)
    
    depth_map_norm = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    
    # Save or display the depth map
    print(f'Saving resulting depth map to {os.path.join(SRC, folder + "-depth_map_" + str(factor) +".png")}')
    cv2.imwrite(os.path.join(SRC, folder + '-depth_map_' + str(factor) +'.png'), depth_map_norm)
    print()

Starting pre-calculations for GX010988\Hnízdo_719_unrolled_final.stl
Desired dimensions of depth map: (830, 1964)


Finding boundaries:   0%|          | 0/15605175 [00:00<?, ?it/s]

Boundaries of scan mesh: (0.0, 202.01604, -0.21254304, 93.31924)
Scaling between mesh and depth map: (9.722000437635671, 8.87398947925416)


Computing depth map for GX010988\Hnízdo_719_unrolled_final.stl:   0%|          | 0/15605175 [00:00<?, ?it/s]

Saving resulting depth map to /Users/fathe/OneDrive/Documents/UK/MFF/Thesis/input\GX010988-depth_map_0.1.png

Starting pre-calculations for GX010996\Hnízdo_732_unrolled_final.stl
Desired dimensions of depth map: (827, 2009)


...

Saving resulting depth map to /Users/fathe/OneDrive/Documents/UK/MFF/Thesis/input\GX010996-depth_map_0.1.png

Starting pre-calculations for GX010998\Hnizdo_718_unrolled_final.stl
Desired dimensions of depth map: (754, 2013)


Finding boundaries:   0%|          | 0/17814390 [00:00<?, ?it/s]

Boundaries of scan mesh: (0.0, 201.56668, -0.24469005, 85.62478)
Scaling between mesh and depth map: (9.98676959371452, 8.780769364384074)


Computing depth map for GX010998\Hnizdo_718_unrolled_final.stl:   0%|          | 0/17814390 [00:00<?, ?it/s]

Saving resulting depth map to /Users/fathe/OneDrive/Documents/UK/MFF/Thesis/input\GX010998-depth_map_0.1.png



In [13]:
path_dm = os.path.join(SRC, "GX010988-depth_map_0.1.png")

In [14]:
depth_map = cv2.imread(path_dm, cv2.IMREAD_GRAYSCALE)

In [23]:
# Define crop indices (adjust these values based on your specific depth map)
y_start = 85  # Starting row (inclusive)
y_end = depth_map.shape[0]-100  # Ending row (exclusive)
print(y_start, y_end)

# Crop the depth map
cropped_depth_map = depth_map[y_start:y_end, :]

cv2.imshow('Inpainted Depth Map', cv2.resize(cropped_depth_map, fx=1, fy=1, dsize=(0,0)))
cv2.waitKey(0)
cv2.destroyAllWindows()

85 727


In [24]:
# Create a binary mask for black pixels (missing data)
# Ensure that you treat values of 0 in the depth map as missing
mask = (cropped_depth_map == 0).astype(np.uint8)  # Set to 255 for missing pixels

# Inpaint the depth map using the mask
depth_map_inpainted = cv2.inpaint(cropped_depth_map, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

# Display or save the inpainted depth map
cv2.imwrite(path_dm.replace(".png", "-inpainted.png"), depth_map_inpainted)
# cv2.imshow('Inpainted Depth Map', depth_map_inpainted)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

True